In [1]:
import pandas as pd
import numpy as np
import nltk
from tqdm.auto import tqdm
from pandarallel import pandarallel
import string
from HanTa import HanoverTagger as ht
import mgzip
import pickle

pandarallel.initialize(progress_bar=True)
tqdm.pandas()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Preprocessing of data sets
This notebook contains the necessary steps for the preprocessing of the data sets. It processes the data sets for both types of data, comments and articles.

## Processing of articles

In [2]:
# load csv of file to process
zeit = pd.read_csv('../data/zeit_scraped.gzip', compression='gzip', low_memory=False, usecols=["title", "date", "combined_text"])
welt = pd.read_csv('../data/welt_scraped.gzip', compression='gzip', low_memory=False, usecols=["title", "date", "combined_text"])
tagesspiegel = pd.read_csv('../data/tagesspiegel_scraped.gzip', compression='gzip', low_memory=False, usecols=["title", "date", "combined_text"])

In [3]:
zeit['newspaper'] = 'zeit'
welt['newspaper'] = 'welt'
tagesspiegel['newspaper'] = 'tagespiegel'

In [4]:
combined_news = pd.concat([zeit, welt, tagesspiegel])

In [5]:
# there was one document that contained a lot of arabic characters, thus it was removed
combined_news = combined_news[combined_news['combined_text'].str.contains('في')==False ]

### Tokenize, remove punctuation & lower casing

In [6]:
combined_news['text_token'] = combined_news['combined_text'].progress_apply(nltk.word_tokenize)

  0%|          | 0/28432 [00:00<?, ?it/s]

In [7]:
punctuation_custom = list(string.punctuation)
punctuation_custom = punctuation_custom + ['„', '“', '–', '•']

def remove_punctuation(text):
    text_list = []
    for word in text:
        for punctuation in punctuation_custom:
            word = word.replace(punctuation, '')
        if not word:
            continue
        text_list.append(word.lower())
    return text_list

In [8]:
combined_news['text_token'] = combined_news['text_token'].progress_apply(remove_punctuation)

  0%|          | 0/28432 [00:00<?, ?it/s]

### Stop word removal, lemmatization of comments & emotion lexicon
Following, stopwords without meaning ('der', 'wo', etc.) are removed from the comments. I edited the sourced stopwords list and removed any negations, since these contain emotions that I want to capture later in the workflow. Afterwards, the remaining tokenized words for each comment are lemmatized, i.e. reduced to their basic form. In order to do this I utilise the 'Hannover Tagger', which also provides part of speech information, although I do not use it (the main reason that I do not use the PoS information is that the utilised German emotion and sentiment lexical use different PoS abbreviations, resulting in no matches).

In [9]:
stopwords = open('../resources/german_stopwords-master/german_stopwords_topic.txt').read().splitlines()

In [10]:
def stop_word_removal(x):
    return list([str(w) for w in x if not w in stopwords])

In [11]:
combined_news['text_token'] = combined_news['text_token'].progress_apply(stop_word_removal)

  0%|          | 0/28432 [00:00<?, ?it/s]

In [12]:
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

In [13]:
def tagger_custom(input):
    tmp_list = []
    for word in input:
        tmp_list.append(tagger.analyze(word)[0].lower())
    return tmp_list

In [14]:
combined_news['text_token'] = combined_news['text_token'].parallel_apply(tagger_custom)

In [15]:
combined_news['combined_text_joined'] = combined_news['text_token'].apply(' '.join)

In [19]:
combined_news_pre = combined_news[['title', 'combined_text_joined', 'text_token', 'date', 'newspaper']]

In [23]:
combined_news_pre = combined_news_pre.drop_duplicates(subset=['date', 'title', 'combined_text_joined'])

In [25]:
# saving preprocessed file
with mgzip.open('../data/combined_news_pre.mgzip', 'wb') as handle:
    pickle.dump(combined_news, handle)

## Processing of comments

In [2]:
tagesspiegel = pd.read_csv('../data/tagesspiegel_scraped.gzip', compression='gzip', low_memory=False)
zeit = pd.read_csv('../data/zeit_scraped.gzip', compression='gzip', low_memory=False)
welt = pd.read_csv('../data/welt_scraped.gzip', compression='gzip', low_memory=False)

In [3]:
# columns initially scraped but not necessary for the analysis anymore are deleted
# the article text is removed since this notebook is only for the sentiment analysis
# the article title serves as the ID of each article, since it is shorter than the article text
tagesspiegel.drop(['Unnamed: 0', 'combined_text', 'link'], axis=1, inplace=True)
zeit.drop(['Tag0', 'Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5', 'combined_text', 'link'], axis=1, inplace=True)
welt.drop(['Tag0', 'Tag1', 'Tag2', 'Tag3', 'combined_text', 'link'], axis=1, inplace=True)

In [4]:
comment_cols_welt = [col for col in welt.columns if 'Comment' in col]
comment_cols_zeit = [col for col in zeit.columns if 'Comment' in col]
comment_cols_tagesspiegel = [col for col in tagesspiegel.columns if 'Comment' in col]

In [5]:
welt = pd.melt(welt, id_vars=['title', 'date'], value_vars=comment_cols_welt)
zeit = pd.melt(zeit, id_vars=['title', 'date'], value_vars=comment_cols_zeit)
tagesspiegel = pd.melt(tagesspiegel, id_vars=['title', 'date'], value_vars=comment_cols_tagesspiegel)

In [6]:
welt.dropna(inplace=True)
zeit.dropna(inplace=True)
tagesspiegel.dropna(inplace=True)

In [7]:
welt['newspaper'] = 'welt'
zeit['newspaper'] = 'zeit'
tagesspiegel['newspaper'] = 'tagespiegel'

In [8]:
combined_comments = pd.concat([zeit, welt, tagesspiegel])

### Tokenization & punctuation removal

In [9]:
combined_comments['tokens'] = combined_comments['value'].progress_apply(nltk.word_tokenize)

  0%|          | 0/1776963 [00:00<?, ?it/s]

In [10]:
punctuation_custom = list(string.punctuation)
punctuation_custom = punctuation_custom + ['„', '“', '–', '•']

def remove_punctuation(text):
    text_list = []
    for word in text:
        for punctuation in punctuation_custom:
            word = word.replace(punctuation, '')
        if not word:
            continue
        text_list.append(word.lower())
    return text_list

In [11]:
combined_comments['tokens'] = combined_comments['tokens'].progress_apply(remove_punctuation)

  0%|          | 0/1776963 [00:00<?, ?it/s]

### Stop word removal & lemmatization
Following, stopwords without meaning ('der', 'wo', etc.) are removed from the comments. I edited the sourced stopwords list and removed any negations, since these contain emotions that I want to capture later in the workflow. Afterwards, the remaining tokenized words for each comment are lemmatized, i.e. reduced to their basic form. In order to do this I utilise the 'Hannover Tagger', which also provides part of speech information, although I do not use it (the main reason that I do not use the PoS information is that the utilised German emotion and sentiment lexical use different PoS abbreviations, resulting in no matches).'

In [12]:
stopwords = open('../resources/german_stopwords-master/german_stopwords_sentiment.txt').read().splitlines()

In [13]:
def stop_word_removal(x):
    return list([w for w in x if not w in stopwords])

In [14]:
combined_comments['tokens'] = combined_comments['tokens'].progress_apply(stop_word_removal)

  0%|          | 0/1776963 [00:00<?, ?it/s]

In [15]:
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

In [16]:
def tagger_custom(input):
    tmp_list = []
    for word in input:
        tmp_list.append(tagger.analyze(word)[0].lower())
    return tmp_list

In [17]:
combined_comments['tokens'] = combined_comments['tokens'].progress_apply(tagger_custom)

  0%|          | 0/1776963 [00:00<?, ?it/s]

In [ ]:
combined_comments_pre = combined_comments.drop_duplicates(subset=['date', 'variable', 'value'])

In [7]:
# saving preprocessed file
with mgzip.open('../data/combined_comments_pre.mgzip', 'wb') as handle:
    pickle.dump(combined_comments_pre, handle)